In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
import scipy.stats as ss
import os
import matplotlib.collections as collections

In [ ]:
def time_parser_pearson_module_affilitor(PyMatrix,NumOfModules,IdOfSub,TBD,OP,SavingAddress,ModulHyp):
    """
    This function will receive the signal matrix and divide it to asked TBD-point windows, calculating Pearson Matrix.
    
    Inputs:
    PyMatrix = A matrix of brain activity with columns as ROI and rows as time points.
    NumOfModules = Number of Modules
    TBD = TimeBlocksDuration, for example 20 means there are 20 time points in each block to calculate the pearson coeff of
    OP = OverlapPercentage, for example 0.25 means each 2 TimeBlock will have 25% of overlap
    SavingAddress = Where to save the output (the folder)
    ModulHyp = The optional prior module assignment we had
    
    Output:
    The function creats folders in your desired address named IdOfSub (string) each containing numpy matrices named
    SampleXFromTimeY_to_Z.npy, where X is the subject num, Y is the starting time point and Z is the last time point (for the
    calculation of Pearson coeff)
    The nupmy matrix is a matrix with the rows as module affiliation ratio (sums to 1 if all positive) and columns as nodes
    """
    ######################################
    ModSize = np.zeros((NumOfModules,1))
    for i in range(1,NumOfModules+1):
        ModSize[i-1,0]  = np.count_nonzero(ModulHyp == i)
    
    ######################################
    w, h = np.shape(PyMatrix)[1], np.shape(PyMatrix)[1];
    print(w,h)
    DataLength = np.shape(PyMatrix)[0]
    #Length of the data (time dimension)
    BlockNum = int((DataLength - int(OP*TBD)) / (TBD - int(OP*TBD))) -1
    #To calculate how many TimeBlocks will exist in out data considering the overlap
    for timeblock in range(0,BlockNum):
        ParsedPearsonMatrix = np.zeros((w,h))
        for i in range(0,w):
            for j in range(0,h):
                ParsedPearsonMatrix[i][j] = ss.pearsonr(PyMatrix[timeblock*TBD - timeblock*(int(OP*TBD)):(timeblock+1)*TBD - timeblock*(int(OP*TBD)),i],PyMatrix[timeblock*TBD - timeblock*(int(OP*TBD)):(timeblock+1)*TBD - timeblock*(int(OP*TBD)),j])[0]
        
        #Now we have a parsed pearson matrix which is needed to be observed as a picture of the system at that time.
        for i in range (0,w):
            ParsedPearsonMatrix[i][i] = 0
        ModulCoeff = np.zeros((NumOfModules,np.shape(PyMatrix)[1]))
        for i in range(0,w):
            for j in range(0,h):
                k = int(ModulHyp[j][0]-1)
                ModulCoeff[k][i] = ModulCoeff[k][i]+((ParsedPearsonMatrix[i][j])**2)**(1./2)
        
        ModulCoeff2 = np.zeros((np.shape(ModulCoeff)[0],np.shape(ModulCoeff)[1]))
        ModulCoeff2  = ModulCoeff
        ModulCoeff2 = ModulCoeff2/ModSize
        
        sums = ModulCoeff.sum(axis=0,keepdims=1); 
        sums[sums==0] = 1
        #This is to avoid dividing by zero
        
        #Cause we have negative numbers and we want to have something between 0 to 1 after division we divide by the positive value (^2^1/2):
        sums2 = ModulCoeff2.sum(axis=0,keepdims=1); 
        sums2[sums2==0] = 1
        
        
        ModulCoeff = ModulCoeff/sums
        ModulCoeff2 = ModulCoeff2/sums2
        #Now we have a matrix with the rows as module affiliation ratio (sums to 1 if all positive) and columns as nodes.
        
        
        if not os.path.isdir(SavingAddress+'/%s/'%(IdOfSub)):
            os.mkdir(SavingAddress +'/%s/'%(IdOfSub))
        initial = timeblock*TBD - timeblock*(int(OP*TBD))
        final = (timeblock+1)*TBD - timeblock*(int(OP*TBD))
        if not os.path.isdir(SavingAddress+'/%s/Normalized'%(IdOfSub)):
            os.mkdir(SavingAddress +'/%s/Normalized'%(IdOfSub))
        np.save(SavingAddress + '/%s/Subject_%s_FromTime%d_to_%d.npy'%(IdOfSub,IdOfSub,initial,final),ModulCoeff)
        np.save(SavingAddress + '/%s/Normalized/Normalized_Subject_%s_FromTime%d_to_%d.npy'%(IdOfSub,IdOfSub,initial,final),ModulCoeff2)
        

In [ ]:
def mat_to_numpy_converter(address,TableName):
    """
    Converts the .mat Matlab matrices to numpy arrays
    """
    
    inputMatrix = scipy.io.loadmat(address)
    PyMatrix = np.array(inputMatrix[TableName])
    #print(np.shape(PyMatrix))
    return PyMatrix

In [ ]:
ModulHypAdd = 'Needed_Address.mat'
TableName = 'a'
ModulHyp = mat_to_numpy_converter(ModulHypAdd,'a')
NumOfModules = len(np.unique(ModulHyp))
SavingAddress = "Needed_Address'
#time_parser_pearson_module_affilitor(ts,15,'PSY00029',15,0.95,SavingAddress,ModulHyp)

In [ ]:
for sub in ids:
    timeseriesadd = "Needed_Address%s.npy"%sub
    ts = np.load(timeseriesadd)
    SavingAddress = "Needed_Address/%s/"%sub
    time_parser_pearson_module_affilitor(ts,15,sub,15,0.95,SavingAddress,ModulHyp)

In [ ]:
SavingAddress ="Needed_Address"
address= 'Needed_Address/15Affiliation.mat'
TableName= 'a'
TBD= 15
OP= 0.95
DataLength= 128
NumOfModules= 15
NumOfSubs= len(ids)
NumofBrainRegions= 246
SubjectSelection= True
SubjectIds = ids
 


NumOfGroupSubs = len(ids)
    
BlockNum = int((DataLength - int(OP*TBD)) / (TBD - int(OP*TBD))) -1
    
#Plot Info:
NumOfPlotsColumns = 5
NumOfPlotsRows = (NumOfSubs//5) +1

#Average over population measures:
AveMat = np.zeros((NumOfModules,BlockNum))
#NumOfChangedROIs is the NumOfSubs*BlockNum-1 matrix which contains the number of nodes that change their 
#affiliation in each time step. A row for each subject.
NumOfChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
NumOfPriorChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
    
    
inputMatrix = scipy.io.loadmat(address)
PriorAffil = np.array(inputMatrix[TableName])
PriorAffil = np.transpose(PriorAffil)
if SubjectSelection == True :
    SubRange = SubjectIds
else:
    SubRange = np.arange(1,NumOfSubs+1)
    
NumOfSubs = len(SubRange)
    
    
for subject in SubRange:
    print(subject)
    print(SubRange.index(subject))
    MatTot = np.zeros((1,NumofBrainRegions))
    NumofSampInt = subject
    WinningModuleTot = np.zeros((1,NumofBrainRegions))
    for timeblock in range(0,BlockNum): 
        initial = timeblock*TBD - timeblock*(int(OP*TBD))
        final = (timeblock+1)*TBD - timeblock*(int(OP*TBD))
        Mat = np.load(SavingAddress +'/%s/%s/Subject_%s_FromTime%d_to_%d.npy'%(NumofSampInt,NumofSampInt,NumofSampInt,initial,final))
        WinningModuleinT = np.zeros((1,NumofBrainRegions))
        for i in range(0,NumofBrainRegions):
            WinningModuleinT[0,i] = np.argmax(Mat[:,i])   
        MatTot = np.concatenate((MatTot,Mat), axis=0)
        WinningModuleTot = np.concatenate((WinningModuleTot,WinningModuleinT), axis=0)

    #WinningModuleTot is a BlockNum*ROINum matrix which shows the the affiliation of all ROIs to the winning modules in
    #time window (BlockNum)
    #This MatTot is the concatenated matrix for all nodes of a single subject for all time windows
    MatTot = np.delete(MatTot, (0), axis=0)
    #print(np.shape(MatTot))
    WinningModuleTot = np.delete(WinningModuleTot, (0), axis=0)

    WinningModuleTot = WinningModuleTot + 1
        
    ##################################################
    #Now That we have WinningModuleTot which is showing us the affiliations over time for each node, we want
    #to track how many of the nodes are changing their affiliation in each step: NumOfChangedROIs = np.zeros(NumOfSubs,BlockNum-1)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (WinningModuleTot[ii+1,jj]):
                Dif = Dif+1
        NumOfChangedROIs[SubRange.index(subject),ii] = Dif    
        ##################################################
    #This is another way of change-calc, were we compare them with the default one at each step.
    #(and not with the previous step)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (PriorAffil[0,jj]):
                Dif = Dif+1
        NumOfPriorChangedROIs[SubRange.index(subject),ii] = Dif
        #################################################
    ModulDistr = np.zeros((NumOfModules,BlockNum))
    #ModulDistr is a 9*13 (NumofModules*Time) matrix where each element (i,j) shows the number of nodes in the i-th module at time j.
    for i in range (0,NumOfModules):
        for j in range (0,BlockNum):
            ModulDistr[i,j] = np.count_nonzero(WinningModuleTot[j,:]==i+1)

    AveMat = np.add(AveMat, ModulDistr)

    x = list(range(1,BlockNum+1))

    labels = []
    for mol in range(1,NumOfModules+1):
        labels.append("Module {0}".format(mol+1))
        
    np.save(SavingAddress + '/%s/%s/ModuleDistrbOfSubject_%s.npy'%(NumofSampInt,NumofSampInt,NumofSampInt),ModulDistr)
        #ax[(subject-1)//NumOfPlotsColumns,(subject-1)%NumOfPlotsColumns].stackplot(x, ModulDistr,labels=labels)
        
    #plt.show()
AveMat = AveMat/NumOfSubs
print(np.shape(AveMat))
print(AveMat)

x = list(range(1,BlockNum+1))
    #####################################
    #Changing population calculation:
    #print(AveMat)
AffilChange = np.zeros((1,BlockNum-1))
    #AffilChange is a row vector showing the total number of changing agents per time steps.
for j in range(0,BlockNum-1):
        #for i in range (0,NumOfModules):
    AffilChange[0,j] = (1/2) * sum(abs(AveMat[:,j] - AveMat[:,j+1]))
    
xdata = np.arange(1,BlockNum)
    #print(AffilChange)
fig, ax = plt.subplots()
plt.plot(xdata,np.transpose(AffilChange))
plt.title('Population of affiliation change per time step')
plt.grid(True)
plt.show()
    
    
#####################################
#NumOfChangedROIs average plotting:
    
AffilChangeSum = NumOfChangedROIs.sum(axis=0)
AffilChangeSum = AffilChangeSum/NumOfSubs
    
xdata = np.arange(3,BlockNum-3)
#print(AffilChangeSum)
fig, ax = plt.subplots(figsize=(20, 5))
plt.plot(xdata,np.transpose(AffilChangeSum[3:BlockNum-3]))
plt.title('change in modular affiliation of regions from each time point to the next averaged across subjects')
plt.xlabel('windows')
#plt.plot(xdata,np.transpose(NumOfChangedROIs[5,:]))
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
    
    
######################################
#Prior Affilchange plotting
PriorAffilChangeSum = NumOfPriorChangedROIs.sum(axis=0)
PriorAffilChangeSum = PriorAffilChangeSum/NumOfSubs
    
xdata = np.arange(1,BlockNum)
#print(AffilChangeSum)
fig, ax = plt.subplots()
plt.plot(xdata,np.transpose(PriorAffilChangeSum))
plt.title('Changing the Affiliation compared to the prior one averaged over subjects')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()

    
####################################
labelsTot = []
for mol in range(1,NumOfModules+1):
    labelsTot.append("Module {0} average pop".format(mol+1))


fig, ax = plt.subplots()
ax.stackplot(x, AveMat,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Changing the Population of Dependencies over Time for All Subject (Average)')
plt.ylabel('Average Dependent Nodes Population')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.xlabel('Time')
plt.show()

##################################
labelsTot = ["Module 1 average pop","Module 4 average pop","Module 14 average pop"]
y1 = AveMat[0,:]
y2 = AveMat[1,:]
y3 = AveMat[2,:]
y4 = AveMat[3,:]
y5 = AveMat[4,:]
y6 = AveMat[5,:]
y7 = AveMat[6,:]
y8 = AveMat[7,:]
y9 = AveMat[8,:]
y10 = AveMat[9,:]
y11 = AveMat[10,:]
y12 = AveMat[11,:]
y13 = AveMat[12,:]
y14 = AveMat[13,:]
y15 = AveMat[14,:]

y = np.vstack([y1, y4, y14])
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Changing the Population of Dependencies over Time for All Subject in Modules BN_anterior_salience(1),BN_dDMN(4),BN_Visuospatial(14)')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.show()


##############################################
#All Modules of the 15 Module system ploted:
##############################################
#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y1)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module One, BN_anterior_salience')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################
###########################
#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y3)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module Three, BN_Basal_Ganglia')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################
###########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y4)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Four, BN_dDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
############################
###########################
#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y5)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module Five, BN_high_Visual')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################
###########################
#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y12)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module Twelve, BN_Sensorimotor')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################
###########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y13)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Thirteen, BN_vDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
############################
###########################
#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y14)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module Fourteen, BN_Visuospatial')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################
###########################

#     fig, ax = plt.subplots(figsize=(20, 5))
#     ax.stackplot(x, y15)
#     plt.title('Changing the Population of Dependencies over Time for All Subject in Module Fifteen, UD')
#     plt.ylabel('Average Dependent Nodes Population')
#     plt.xlabel('Time')
#     plt.grid(color='b', linestyle=':', linewidth=0.2)
#     plt.show()
############################

In [ ]:
#Average Plotter
#ModulHyp is the pre-specified modules we have intruduced   
ModulHypAdd = 'Needed_Address/15Affiliation.mat'
TableName = 'a'
ModulHyp = mat_to_numpy_converter(ModulHypAdd,'a')
NumOfModules = len(np.unique(ModulHyp))
TBD = 15
OP = 0.95
DataLength = 128

SavingAddress ="Needed_Address"
NumOfGroupSubs = len(ids) - 100
# all_subjects_average_plotter(SavingAddress,ModulHypAdd,TableName,TBD,OP,DataLength,
#                                    NumOfModules,NumOfGroupSubs,246,
#                                    SubjectSelection = True,SubjectIds=ids)
#Jess.all_subjects_average_plotter(SavingAddress,address,TableName,15,0.95,128,15,176,246,True,SubjectIds)

In [ ]:
ids = ['subject_a','subject_b','subject_c']

# Non_Normalized

In [ ]:
###Variables######################
SavingAddress ="Needed_Address"
address= 'Needed_Address/15Affiliation.mat'
TableName= 'a'
TBD= 15
OP= 0.95
DataLength= 128
NumOfModules= 15
NumOfSubs= len(ids)
NumofBrainRegions= 246
SubjectSelection= True
SubjectIds = ids
NumOfGroupSubs = len(ids)
###################################



    
BlockNum = int((DataLength - int(OP*TBD)) / (TBD - int(OP*TBD))) -1
    
#Plot Info:
NumOfPlotsColumns = 5
NumOfPlotsRows = (NumOfSubs//5) +1


#Average over population measures:
AveMat = np.zeros((NumOfModules,BlockNum))
#NumOfChangedROIs is the NumOfSubs*BlockNum-1 matrix which contains the number of nodes that change their 
#affiliation in each time step. A row for each subject.
NumOfChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
NumOfPriorChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
    
    
inputMatrix = scipy.io.loadmat(address)
PriorAffil = np.array(inputMatrix[TableName])
PriorAffil = np.transpose(PriorAffil)
if SubjectSelection == True :
    SubRange = SubjectIds
else:
    SubRange = np.arange(1,NumOfSubs+1)
    
NumOfSubs = len(SubRange)
    
    
for subject in SubRange:
    MatTot = np.zeros((1,NumofBrainRegions))
    NumofSampInt = subject
    WinningModuleTot = np.zeros((1,NumofBrainRegions))
    for timeblock in range(0,BlockNum): 
        initial = timeblock*TBD - timeblock*(int(OP*TBD))
        final = (timeblock+1)*TBD - timeblock*(int(OP*TBD))
        Mat = np.load(SavingAddress +'/%s/%s/Subject_%s_FromTime%d_to_%d.npy'%(NumofSampInt,NumofSampInt,NumofSampInt,initial,final))
        WinningModuleinT = np.zeros((1,NumofBrainRegions))
        for i in range(0,NumofBrainRegions):
            WinningModuleinT[0,i] = np.argmax(Mat[:,i])   
        MatTot = np.concatenate((MatTot,Mat), axis=0)
        WinningModuleTot = np.concatenate((WinningModuleTot,WinningModuleinT), axis=0)

    #WinningModuleTot is a BlockNum*ROINum matrix which shows the the affiliation of all ROIs to the winning modules in
    #time window (BlockNum)
    #This MatTot is the concatenated matrix for all nodes of a single subject for all time windows
    MatTot = np.delete(MatTot, (0), axis=0)
    #print(np.shape(MatTot))
    WinningModuleTot = np.delete(WinningModuleTot, (0), axis=0)

    WinningModuleTot = WinningModuleTot + 1
 
    ##################################################
    #Now That we have WinningModuleTot which is showing us the affiliations over time for each node, we want
    #to track how many of the nodes are changing their affiliation in each step: NumOfChangedROIs = np.zeros(NumOfSubs,BlockNum-1)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (WinningModuleTot[ii+1,jj]):
                Dif = Dif+1
        NumOfChangedROIs[SubRange.index(subject),ii] = Dif    
        ##################################################
    #This is another way of change-calc, were I compare them with the default one at each step.
    #(and not with the previous step)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (PriorAffil[0,jj]):
                Dif = Dif+1
        NumOfPriorChangedROIs[SubRange.index(subject),ii] = Dif
        #################################################
    ModulDistr = np.zeros((NumOfModules,BlockNum))
    #print(np.shape(ModulDistr))
    #ModulDistr is a 9*13 (NumofModules*Time) matrix where each element (i,j) shows the number of nodes in the i-th module at time j.
    for i in range (0,NumOfModules):
        for j in range (0,BlockNum):
            ModulDistr[i,j] = np.count_nonzero(WinningModuleTot[j,:]==i+1)

    AveMat = np.add(AveMat, ModulDistr)

    #print(ModulDistr)
    x = list(range(1,BlockNum+1))

    labels = []
    for mol in range(1,NumOfModules+1):
        labels.append("Module {0}".format(mol+1))
        
    np.save(SavingAddress + '/%s/%s/ModuleDistrbOfSubject_%s.npy'%(NumofSampInt,NumofSampInt,NumofSampInt),ModulDistr)
        #ax[(subject-1)//NumOfPlotsColumns,(subject-1)%NumOfPlotsColumns].stackplot(x, ModulDistr,labels=labels)
        
    #plt.show()
AveMat = AveMat/NumOfSubs
# print(np.shape(AveMat))
# print(AveMat)

x = list(range(1,BlockNum+1))
    #####################################
    #Changing population calculation:
    #print(AveMat)
AffilChange = np.zeros((1,BlockNum-1))
    #AffilChange is a row vector showing the total number of changing agents per time steps.
for j in range(0,BlockNum-1):
        #for i in range (0,NumOfModules):
    AffilChange[0,j] = (1/2) * sum(abs(AveMat[:,j] - AveMat[:,j+1]))
    
# xdata = np.arange(1,BlockNum)
#     #print(AffilChange)
# fig, ax = plt.subplots()
# plt.plot(xdata,np.transpose(AffilChange))
# plt.title('Population of affiliation change per time step')
# plt.grid(True)
# plt.show()
    
    
    #####################################
    #NumOfChangedROIs average plotting:
    
AffilChangeSum = NumOfChangedROIs.sum(axis=0)
AffilChangeSum = AffilChangeSum/NumOfSubs
    
    

xdata = np.arange(3,BlockNum-3)
#print(AffilChangeSum)

fig, ax = plt.subplots(figsize=(20, 5))
# collection = collections.BrokenBarHCollection.span_where(xdata, ymin=0, ymax=100, where= np.transpose(AffilChangeSum[3:BlockNum-3])> 0, facecolor='green', alpha=0.5)
# ax.add_collection(collection)



plt.plot(xdata,np.transpose(AffilChangeSum[3:BlockNum-3])/246)
plt.title('change in modular affiliation of regions from each time point to the next averaged across subjects')
plt.xlabel('windows')
plt.ylabel('Flexibility')
#plt.plot(xdata,np.transpose(NumOfChangedROIs[5,:]))
plt.grid(color='b', linestyle=':', linewidth=0.2)

for i in range (0,15):
    plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
    if i%4==0:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min((AffilChangeSum[3:BlockNum-3])/246), "0back", fontdict=None)
    if i%4==1:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
    if i%4==2:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min((AffilChangeSum[3:BlockNum-3])/246), "2back", fontdict=None)
    if i%4==3:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)

plt.show()
    
    
######################################
#Prior Affilchange plotting
PriorAffilChangeSum = NumOfPriorChangedROIs.sum(axis=0)
PriorAffilChangeSum = PriorAffilChangeSum/NumOfSubs
    
xdata = np.arange(1,BlockNum)
#print(AffilChangeSum)
fig, ax = plt.subplots(figsize=(20, 5))
plt.plot(xdata,np.transpose(PriorAffilChangeSum)/246)
plt.title('Changing the Affiliation compared to the prior one averaged over subjects')
#plt.plot(xdata,np.transpose(NumOfChangedROIs[5,:]))
plt.grid(color='b', linestyle=':', linewidth=0.2)

for i in range (0,15):
    plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
    if i%4==0:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min(np.transpose(PriorAffilChangeSum)/246), "0back", fontdict=None)
    if i%4==1:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
    if i%4==2:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min(np.transpose(PriorAffilChangeSum)/246), "2back", fontdict=None)
    if i%4==3:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)

plt.show()

    
    ####################################
labelsTot = []
for mol in range(1,NumOfModules+1):
    labelsTot.append("Module {0} average pop".format(mol+1))


fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, AveMat,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Changing the Population of Dependencies over Time for All Subject (Average)')
plt.ylabel('Average Dependent Nodes Population')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.xlabel('Time')
plt.show()

##################################
labelsTot = ["Module 1 average pop","Module 4 average pop","Module 14 average pop"]
y1 = AveMat[0,:]
y2 = AveMat[1,:]
y3 = AveMat[2,:]
y4 = AveMat[3,:]
y5 = AveMat[4,:]
y6 = AveMat[5,:]
y7 = AveMat[6,:]
y8 = AveMat[7,:]
y9 = AveMat[8,:]
y10 = AveMat[9,:]
y11 = AveMat[10,:]
y12 = AveMat[11,:]
y13 = AveMat[12,:]
y14 = AveMat[13,:]
y15 = AveMat[14,:]

y = np.vstack([y1, y4, y14])
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Changing the Population of Dependencies over Time for All Subject in Modules BN_anterior_salience(1),BN_dDMN(4),BN_Visuospatial(14)')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.show()


##############################################
#All Modules of the 15 Module system ploted:
##############################################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y1)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module One, BN_anterior_salience')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y3)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Three, BN_Basal_Ganglia')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y4)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Four, BN_dDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y5)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Five, BN_high_Visual')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y12)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Twelve, BN_Sensorimotor')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y13)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Thirteen, BN_vDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y14)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Fourteen, BN_Visuospatial')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################

fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y15)
plt.title('Changing the Population of Dependencies over Time for All Subject in Module Fifteen, UD')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################

# Normalized:

In [ ]:
###Variables######################
SavingAddress ="Needed_Address"
address= 'Needed_Address/15Affiliation.mat'
TableName= 'a'
TBD= 15
OP= 0.95
DataLength= 128
NumOfModules= 15
NumOfSubs= len(lis)
NumofBrainRegions= 246
SubjectSelection= True
SubjectIds = lis
NumOfGroupSubs = len(lis)
###################################

BlockNum = int((DataLength - int(OP*TBD)) / (TBD - int(OP*TBD))) -1
    
#Plot Info:
NumOfPlotsColumns = 5
NumOfPlotsRows = (NumOfSubs//5) +1


#Average over population measures:
AveMat = np.zeros((NumOfModules,BlockNum))
#NumOfChangedROIs is the NumOfSubs*BlockNum-1 matrix which contains the number of nodes that change their 
#affiliation in each time step. A row for each subject.
NumOfChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
NumOfPriorChangedROIs = np.zeros((NumOfSubs,BlockNum-1))
    
    
inputMatrix = scipy.io.loadmat(address)
PriorAffil = np.array(inputMatrix[TableName])
PriorAffil = np.transpose(PriorAffil)
if SubjectSelection == True :
    SubRange = SubjectIds
else:
    SubRange = np.arange(1,NumOfSubs+1)
    
NumOfSubs = len(SubRange)
    
    
for subject in SubRange:
    MatTot = np.zeros((1,NumofBrainRegions))
    NumofSampInt = subject
    WinningModuleTot = np.zeros((1,NumofBrainRegions))
    for timeblock in range(0,BlockNum): 
        initial = timeblock*TBD - timeblock*(int(OP*TBD))
        final = (timeblock+1)*TBD - timeblock*(int(OP*TBD))
        Mat = np.load(SavingAddress +'/%s/%s/Normalized/Normalized_Subject_%s_FromTime%d_to_%d.npy'%(NumofSampInt,NumofSampInt,NumofSampInt,initial,final))
        WinningModuleinT = np.zeros((1,NumofBrainRegions))
        for i in range(0,NumofBrainRegions):
            WinningModuleinT[0,i] = np.argmax(Mat[:,i])   
        MatTot = np.concatenate((MatTot,Mat), axis=0)
        WinningModuleTot = np.concatenate((WinningModuleTot,WinningModuleinT), axis=0)

    #WinningModuleTot is a BlockNum*ROINum matrix which shows the the affiliation of all ROIs to the winning modules in
    #time window (BlockNum)
    #This MatTot is the concatenated matrix for all nodes of a single subject for all time windows
    MatTot = np.delete(MatTot, (0), axis=0)
    #print(np.shape(MatTot))
    WinningModuleTot = np.delete(WinningModuleTot, (0), axis=0)

    WinningModuleTot = WinningModuleTot + 1
    #print(np.shape(WinningModuleTot))
    #print(WinningModuleTot)
        
    ##################################################
    #Now That we have WinningModuleTot which is showing us the affiliations over time for each node, we want
    #to track how many of the nodes are changing their affiliation in each step: NumOfChangedROIs = np.zeros(NumOfSubs,BlockNum-1)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (WinningModuleTot[ii+1,jj]):
                Dif = Dif+1
        NumOfChangedROIs[SubRange.index(subject),ii] = Dif    
        ##################################################
    #This is another way of change-calc, were I compare them with the default one at each step.
    #(and not with the previous step)
    for ii in range(0,BlockNum-1):
        Dif = 0
        for jj in range (0, NumofBrainRegions):
            if (WinningModuleTot[ii,jj]) != (PriorAffil[0,jj]):
                Dif = Dif+1
        NumOfPriorChangedROIs[SubRange.index(subject),ii] = Dif
        #################################################
    ModulDistr = np.zeros((NumOfModules,BlockNum))
    #print(np.shape(ModulDistr))
    #ModulDistr is a 9*13 (NumofModules*Time) matrix where each element (i,j) shows the number of nodes in the i-th module at time j.
    for i in range (0,NumOfModules):
        for j in range (0,BlockNum):
            ModulDistr[i,j] = np.count_nonzero(WinningModuleTot[j,:]==i+1)

    AveMat = np.add(AveMat, ModulDistr)

    #print(ModulDistr)
    x = list(range(1,BlockNum+1))

    labels = []
    for mol in range(1,NumOfModules+1):
        labels.append("Module {0}".format(mol+1))
        
    #np.save(SavingAddress + '/%s/%s/Normalized/Normalized_ModuleDistrbOfSubject_%s.npy'%(NumofSampInt,NumofSampInt,NumofSampInt),ModulDistr)
        #ax[(subject-1)//NumOfPlotsColumns,(subject-1)%NumOfPlotsColumns].stackplot(x, ModulDistr,labels=labels)
        
    #plt.show()
AveMat = AveMat/NumOfSubs
# print(np.shape(AveMat))
# print(AveMat)

x = list(range(1,BlockNum+1))
    #####################################
    #Changing population calculation:
    #print(AveMat)
AffilChange = np.zeros((1,BlockNum-1))
    #AffilChange is a row vector showing the total number of changing agents per time steps.
for j in range(0,BlockNum-1):
        #for i in range (0,NumOfModules):
    AffilChange[0,j] = (1/2) * sum(abs(AveMat[:,j] - AveMat[:,j+1]))
    
# xdata = np.arange(1,BlockNum)
#     #print(AffilChange)
# fig, ax = plt.subplots()
# plt.plot(xdata,np.transpose(AffilChange))
# plt.title('Population of affiliation change per time step')
# plt.grid(True)
# plt.show()
    
    
    #####################################
    #NumOfChangedROIs average plotting:
    
AffilChangeSum = NumOfChangedROIs.sum(axis=0)
AffilChangeSum = AffilChangeSum/NumOfSubs
    
    

xdata = np.arange(3,BlockNum-3)
#print(AffilChangeSum)

fig, ax = plt.subplots(figsize=(20, 5))
# collection = collections.BrokenBarHCollection.span_where(xdata, ymin=0, ymax=100, where= np.transpose(AffilChangeSum[3:BlockNum-3])> 0, facecolor='green', alpha=0.5)
# ax.add_collection(collection)



plt.plot(xdata,np.transpose(AffilChangeSum[3:BlockNum-3]/246))
plt.title('Normalized, change in modular affiliation of regions from each time point to the next averaged across subjects')
plt.xlabel('windows', fontsize=22)
plt.ylabel('Flexibility', fontsize=22)
matplotlib.rc('font', size=10)
plt.grid(color='b', linestyle=':', linewidth=0.2)



                       
for i in range (0,15):
    plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
    if i%4==0:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min(AffilChangeSum[3:BlockNum-3]/246), "0back", fontdict=None)
    if i%4==1:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
    if i%4==2:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25),  min(AffilChangeSum[3:BlockNum-3]/246), "2back", fontdict=None)
    if i%4==3:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)

ax.tick_params(axis="y", labelsize=5)
plt.show()
    
    
######################################
#Prior Affilchange plotting
PriorAffilChangeSum = NumOfPriorChangedROIs.sum(axis=0)
PriorAffilChangeSum = PriorAffilChangeSum/NumOfSubs
    
xdata = np.arange(1,BlockNum)
#print(AffilChangeSum)
fig, ax = plt.subplots(figsize=(20, 5))
plt.plot(xdata,np.transpose(PriorAffilChangeSum)/246)
plt.title('Normalized, Changing the Affiliation compared to the prior one averaged over subjects')
#plt.plot(xdata,np.transpose(NumOfChangedROIs[5,:]))
plt.grid(color='b', linestyle=':', linewidth=0.2)

for i in range (0,15):
    plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
    if i%4==0:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min(np.transpose(PriorAffilChangeSum)/246), "0back", fontdict=None)
    if i%4==1:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
    if i%4==2:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
        matplotlib.pyplot.text(7.6*(i+0.25), min(np.transpose(PriorAffilChangeSum)/246), "2back", fontdict=None)
    if i%4==3:
        plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)


plt.show()

    
    ####################################
labelsTot = []
for mol in range(1,NumOfModules+1):
    labelsTot.append("Module {0} average pop".format(mol+1))


fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, AveMat,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject (Average)')
plt.ylabel('Average Dependent Nodes Population')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.xlabel('Time Windows')
plt.show()

##################################
labelsTot = ["Module 1 average pop","Module 4 average pop","Module 14 average pop"]
y1 = AveMat[0,:]
y2 = AveMat[1,:]
y3 = AveMat[2,:]
y4 = AveMat[3,:]
y5 = AveMat[4,:]
y6 = AveMat[5,:]
y7 = AveMat[6,:]
y8 = AveMat[7,:]
y9 = AveMat[8,:]
y10 = AveMat[9,:]
y11 = AveMat[10,:]
y12 = AveMat[11,:]
y13 = AveMat[12,:]
y14 = AveMat[13,:]
y15 = AveMat[14,:]

y = np.vstack([y1, y4, y14])
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y,labels=labelsTot)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Modules BN_anterior_salience(1),BN_dDMN(4),BN_Visuospatial(14)')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.show()


##############################################
#All Modules of the 15 Module system ploted:
##############################################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y1)
plt.title('Normalized,,Changing the Population of Dependencies over Time for All Subject in Module One, BN_anterior_salience')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y3)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Three, BN_Basal_Ganglia')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y4)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Four, BN_dDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
# for i in range (0,15):
#     plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
#     if i%4==0:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
#         matplotlib.pyplot.text(7.6*(i+0.25), 10, "0back", fontdict=None)
#     if i%4==1:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
#     if i%4==2:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
#         matplotlib.pyplot.text(7.6*(i+0.25), 10, "2back", fontdict=None)
#     if i%4==3:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)


plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y5)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Five, BN_high_Visual')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y12)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Twelve, BN_Sensorimotor')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y13)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Thirteen, BN_vDMN')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y14)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Fourteen, BN_Visuospatial')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)

# for i in range (0,15):
#     plt.axvline(7.6*(i+1), linestyle='dashed',color='grey')
#     if i%4==0:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFD0E6', alpha=0.5)
#         matplotlib.pyplot.text(7.6*(i+0.25), 10, "0back", fontdict=None)
#     if i%4==1:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)
#     if i%4==2:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#BFE6CA', alpha=0.5)
#         matplotlib.pyplot.text(7.6*(i+0.25), 10, "2back", fontdict=None)
#     if i%4==3:
#         plt.axvspan(7.6*i, 7.6*(i+1), facecolor='#FFFFFF', alpha=0.5)




plt.show()
###########################
##########################
fig, ax = plt.subplots(figsize=(20, 5))
ax.stackplot(x, y15)
plt.title('Normalized, Changing the Population of Dependencies over Time for All Subject in Module Fifteen, UD')
plt.ylabel('Average Dependent Nodes Population')
plt.xlabel('Time Windows')
plt.grid(color='b', linestyle=':', linewidth=0.2)
plt.show()
###########################